In [ ]:
import json
from collections import defaultdict



input_file = "context_target_exprs_K20_scored_vicuna_v15_13b.json"

with open(input_file, 'r') as f:
    scored_exprs = json.load(f)
# category 0: (subject, action) --> place
# category 1: (subject, place) --> action

subject_action_t20 = defaultdict(list)
subject_place_t20 = defaultdict(list)

for expr_dict in scored_exprs:
    if expr_dict["category"] == 0:
        if expr_dict["action"] not in subject_action_t20[expr_dict["subject"]]:
            subject_action_t20[expr_dict["subject"]].append(expr_dict["action"])
    elif expr_dict["category"] == 1:
        if expr_dict["place"] not in subject_place_t20[expr_dict["subject"]]:
            subject_place_t20[expr_dict["subject"]].append(expr_dict["place"])

In [ ]:
subject_action_t20 = {}
for subj, val_list in subject_action_t20.items():
    assert len(val_list) == 20
    subject_action_t20[subj] = val_list[:20]
    # for action in val_list:
    #     print(f"{subj} {action}")
    # print("="*30)
with open("subject_action_t20_to_clean.json", "w") as f:
    json.dump(subject_action_t20, f, indent=2)
with open("subject_action_t20_cleaned.json", "w") as f:
    json.dump(subject_action_t20, f, indent=2)

In [ ]:
file_path = "subject_action_T20_cleaned.json"
with open(file_path, "r") as f:
    filtered_subject_action_tuples = json.load(f)

length = 0
for subject in filtered_subject_action_tuples:
    length += len(filtered_subject_action_tuples[subject])
print(f"Total tuples: {length}")

In [ ]:
P = 20
def nested_dict():
    return defaultdict(list)

# Create the nested defaultdict
place_to_subject_action = defaultdict(nested_dict)

for expr_dict in scored_exprs:
    if expr_dict["category"] == 0:
        if expr_dict["subject"] in filtered_subject_action_tuples and expr_dict["action"] in filtered_subject_action_tuples[expr_dict["subject"]]:
            place_to_subject_action[expr_dict["subject"]][expr_dict["action"]].append([expr_dict["place"], expr_dict["logprob"]])

for subject in place_to_subject_action:
    for action in place_to_subject_action[subject]:
        place_to_subject_action[subject][action] = sorted(place_to_subject_action[subject][action], key=lambda x: x[1], reverse=False)[:P] # last p items

out_1 = f"subject_T20action_B{P}place.json"
out_2 = f"subject_T20action_B{P}place_cleaned.json"

with open(out_1, 'w') as f:
    json.dump(place_to_subject_action, f, indent=2)
with open(out_2, 'w') as f:
    json.dump(place_to_subject_action, f, indent=2)

In [ ]:
with open("/home/liu/github_repos/MMChallenger/gen_mm_input/subject_T20action_B20place_cleaned.json", "r") as f:
    data = json.load(f)

length = 0
for subject in data:
    for action in data[subject]:
        for place in data[subject][action]:
            length += 1
print(length)

In [ ]:
for subj, val_list in subject_place_t20.items():
    assert len(val_list) == 20
    subject_place_t20[subj] = val_list[:5]

with open("subject_place_T5_to_clean.json", "w") as f:
    json.dump(subject_place_t20, f, indent=2)
with open("subject_place_T5_cleaned.json", "w") as f:
    json.dump(subject_place_t20, f, indent=2)

In [ ]:
file_path = "subject_place_T5_cleaned.json"
with open(file_path, "r") as f:
    filtered_subject_place_tuples = json.load(f)

length = 0
for subject in filtered_subject_place_tuples:
    length += len(filtered_subject_place_tuples[subject])
print(f"Total tuples: {length}")

In [ ]:
# Create the nested defaultdict
action_to_subject_place = defaultdict(nested_dict)

for expr_dict in scored_exprs:
    if expr_dict["category"] == 1:
        if expr_dict["subject"] in filtered_subject_place_tuples and expr_dict["place"] in filtered_subject_place_tuples[expr_dict["subject"]]:
            action_to_subject_place[expr_dict["subject"]][expr_dict["place"]].append([expr_dict["action"], expr_dict["logprob"]])

for subject in action_to_subject_place:
    for place in action_to_subject_place[subject]:
        action_to_subject_place[subject][place] = sorted(action_to_subject_place[subject][place], key=lambda x: x[1], reverse=False)[:P] # last p items

out_1 = f"subject_T20place_B{P}action.json"
out_2 = f"subject_T20place_B{P}action_cleaned.json"

with open(out_1, 'w') as f:
    json.dump(action_to_subject_place, f, indent=2)
with open(out_2, 'w') as f:
    json.dump(action_to_subject_place, f, indent=2)

In [ ]:
baby_on_bed = []

for expr_dict in scored_exprs:
    if expr_dict["category"] == 1:
        if expr_dict["subject"] == "a gardener" and expr_dict["place"] == "in a garden":
            baby_on_bed.append([expr_dict["action"], expr_dict["logprob"]])

baby_on_bed = sorted(baby_on_bed, key=lambda x: x[1], reverse=False)
baby_on_bed